# Face Mask Recognition 

### 1.Import necessary packages

In [8]:
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout
from keras.models import Model, load_model
from keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import numpy as np

### 2.Building the Model

In [15]:
model = Sequential([
    Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2,2),
    
    Conv2D(100, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dropout(0.5),
    Dense(50, activation='relu'),
    Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

### 3.Inserting the Dataset

In [7]:
TRAINING_DIR = "./Dataset/train"
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(TRAINING_DIR, 
                                                    batch_size=10, 
                                                    target_size=(150, 150))
VALIDATION_DIR = "./Dataset/test"
validation_datagen = ImageDataGenerator(rescale=1.0/255)

validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, 
                                                         batch_size=10, 
                                                         target_size=(150, 150))

Found 1315 images belonging to 2 classes.
Found 194 images belonging to 2 classes.


### 4.Creating Checkpoint

In [5]:
checkpoint = ModelCheckpoint('model2-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

### 5.Training the Model

In [22]:
history = model.fit_generator(train_generator,
                              epochs=11,
                              validation_data=validation_generator,
                              callbacks=[checkpoint])

Epoch 1/11
132/132 [==============================] - 52s 393ms/step - loss: 0.5479 - acc: 0.7749 - val_loss: 0.3379 - val_acc: 0.8505
Epoch 2/11
132/132 [==============================] - 51s 384ms/step - loss: 0.3875 - acc: 0.8563 - val_loss: 0.2135 - val_acc: 0.9227
Epoch 3/11
132/132 [==============================] - 54s 412ms/step - loss: 0.2987 - acc: 0.8897 - val_loss: 0.1275 - val_acc: 0.9588
Epoch 4/11
132/132 [==============================] - 57s 433ms/step - loss: 0.2883 - acc: 0.9049 - val_loss: 0.0944 - val_acc: 0.9691
Epoch 5/11
132/132 [==============================] - 59s 444ms/step - loss: 0.2541 - acc: 0.8951 - val_loss: 0.1637 - val_acc: 0.9227
Epoch 6/11
132/132 [==============================] - 58s 441ms/step - loss: 0.2201 - acc: 0.9141 - val_loss: 0.1315 - val_acc: 0.9381
Epoch 7/11
132/132 [==============================] - 58s 438ms/step - loss: 0.1999 - acc: 0.9308 - val_loss: 0.1233 - val_acc: 0.9381
Epoch 8/11
132/132 [==============================] - 5

### 6.Testing the Model with OpenCV

In [14]:
import cv2
import numpy as np
from keras.models import load_model
model=load_model(r"C:\Users\calm\model2-009.model")

results={1:'without mask',0:'mask'}
GR_dict={1:(0,0,255),0:(0,255,0)}

rect_size = 4
cap = cv2.VideoCapture(0) 


haarcascade = cv2.CascadeClassifier(r'C:\Users\calm\anaconda3\envs\tensorflow\Lib\site-packages\cv2\data\haarcascade_frontalface_default.xml')

while True:
    (rval, im) = cap.read()
    im=cv2.flip(im,1,1) 

    
    rerect_size = cv2.resize(im, (im.shape[1] // rect_size, im.shape[0] // rect_size))
    faces = haarcascade.detectMultiScale(rerect_size)
    for f in faces:
        (x, y, w, h) = [v * rect_size for v in f] 
        
        face_img = im[y:y+h, x:x+w]
        rerect_sized=cv2.resize(face_img,(150,150))
        normalized=rerect_sized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)

        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),GR_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),GR_dict[label],-1)
        cv2.putText(im, results[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)

    cv2.imshow('LIVE',   im)
    key = cv2.waitKey(10)
    
    if key == ord('a'): 
        break

cap.release()

cv2.destroyAllWindows()